# Multi-Cloud Hybrid Infrastructure

## Гибридное развертывание: AWS + GCP + On-Premise

Этот туториал демонстрирует создание гибридной мульти-облачной инфраструктуры с использованием:

- **AWS**: Primary cloud (Frontend, API Gateway)
- **GCP**: Secondary cloud (Data Processing, ML Services)
- **On-Premise**: Legacy systems, databases
- **Tailscale**: Mesh VPN для связи между облаками
- **Terraform Workspaces**: Управление окружениями

### Архитектура

```
┌─────────────────────────────────────────────────────┐
│                  Tailscale Mesh VPN                 │
│              (Secure Cross-Cloud Network)           │
└──────────┬──────────────────┬──────────────┬────────┘
           │                  │              │
    ┌──────▼──────┐   ┌───────▼──────┐  ┌───▼────────┐
    │     AWS     │   │     GCP      │  │ On-Premise │
    │             │   │              │  │            │
    │ - Frontend  │   │ - BigQuery   │  │ - Legacy   │
    │ - API GW    │   │ - Dataflow   │  │   DB       │
    │ - RDS       │   │ - AI/ML      │  │ - File     │
    │ - S3        │   │ - GCS        │  │   Storage  │
    └─────────────┘   └──────────────┘  └────────────┘
```

## Шаг 1: Настройка Terraform для Multi-Cloud

In [ ]:
%%writefile main.tf

terraform {
  required_version = ">= 1.5.0"
  
  required_providers {
    aws = {
      source  = "hashicorp/aws"
      version = "~> 5.0"
    }
    google = {
      source  = "hashicorp/google"
      version = "~> 5.0"
    }
  }
  
  backend "s3" {
    bucket         = "multicloud-terraform-state"
    key            = "multicloud/terraform.tfstate"
    region         = "us-east-1"
    encrypt        = true
    dynamodb_table = "terraform-lock"
  }
}

# AWS Provider
provider "aws" {
  region = var.aws_region
  
  default_tags {
    tags = {
      Project     = "MultiCloud"
      ManagedBy   = "Terraform"
      Environment = terraform.workspace
    }
  }
}

# GCP Provider
provider "google" {
  project = var.gcp_project_id
  region  = var.gcp_region
}

## Шаг 2: Создание Tailscale Mesh Network

In [ ]:
%%writefile modules/tailscale-mesh/main.tf

# AWS Tailscale Node
module "tailscale_aws" {
  source = "git::https://github.com/v-grand/infra-network.git//modules/tailscale"

  vpc_id            = module.aws_vpc.vpc_id
  subnet_id         = module.aws_vpc.private_subnets[0]
  tailscale_authkey = var.tailscale_authkey
  
  instance_type = "t3.micro"
  
  tags = {
    Name     = "tailscale-aws-gateway"
    Cloud    = "AWS"
    Location = var.aws_region
  }
  
  # Enable subnet routing
  advertise_routes = [
    module.aws_vpc.vpc_cidr_block
  ]
}

# GCP Tailscale Node
resource "google_compute_instance" "tailscale_gcp" {
  name         = "tailscale-gcp-gateway"
  machine_type = "e2-micro"
  zone         = "${var.gcp_region}-a"

  boot_disk {
    initialize_params {
      image = "ubuntu-os-cloud/ubuntu-2204-lts"
    }
  }

  network_interface {
    network    = google_compute_network.vpc.name
    subnetwork = google_compute_subnetwork.private.name
  }

  metadata_startup_script = <<-EOF
    #!/bin/bash
    curl -fsSL https://tailscale.com/install.sh | sh
    echo 'net.ipv4.ip_forward = 1' | tee -a /etc/sysctl.conf
    sysctl -p
    tailscale up --authkey=${var.tailscale_authkey} --advertise-routes=${google_compute_subnetwork.private.ip_cidr_range}
  EOF

  tags = ["tailscale", "gateway"]
}

# On-Premise Tailscale (manual setup instructions)
output "onprem_tailscale_setup" {
  value = <<-EOT
    To setup Tailscale on on-premise server:
    
    1. Install Tailscale:
       curl -fsSL https://tailscale.com/install.sh | sh
    
    2. Enable IP forwarding:
       echo 'net.ipv4.ip_forward = 1' | sudo tee -a /etc/sysctl.conf
       sudo sysctl -p
    
    3. Connect to Tailscale:
       sudo tailscale up --authkey=${var.tailscale_authkey} --advertise-routes=192.168.1.0/24
    
    4. Approve routes in Tailscale admin console
  EOT
}

## Шаг 3: AWS Infrastructure

In [ ]:
%%writefile modules/aws/main.tf

# VPC
module "aws_vpc" {
  source = "git::https://github.com/v-grand/infra-network.git//modules/vpc"

  project_name = "multicloud-aws"
  environment  = terraform.workspace
  vpc_cidr     = "10.0.0.0/16"

  azs             = ["us-east-1a", "us-east-1b"]
  private_subnets = ["10.0.1.0/24", "10.0.2.0/24"]
  public_subnets  = ["10.0.101.0/24", "10.0.102.0/24"]

  enable_nat_gateway = true
}

# Frontend (S3 + CloudFront)
module "frontend" {
  source = "git::https://github.com/v-grand/infra-aws.git//modules/s3"

  bucket_name = "multicloud-frontend-${terraform.workspace}"
  
  website_configuration = {
    index_document = "index.html"
    error_document = "index.html"
  }
}

# API Gateway (ECS Fargate)
module "api_gateway" {
  source = "git::https://github.com/v-grand/infra-aws.git//modules/ecs"

  cluster_name    = "multicloud-api"
  vpc_id          = module.aws_vpc.vpc_id
  private_subnets = module.aws_vpc.private_subnets
  public_subnets  = module.aws_vpc.public_subnets

  services = {
    api = {
      cpu    = 512
      memory = 1024
      image  = "${var.ecr_registry}/api-gateway:latest"
      port   = 8000
      
      environment = [
        {
          name  = "GCP_ENDPOINT"
          value = "http://100.64.1.10:8080"  # Tailscale IP
        },
        {
          name  = "ONPREM_DB"
          value = "postgresql://100.64.1.20:5432/legacy"  # Tailscale IP
        }
      ]
    }
  }
}

# RDS Database
module "rds" {
  source = "git::https://github.com/v-grand/infra-aws.git//modules/rds"

  identifier     = "multicloud-db"
  engine         = "postgres"
  engine_version = "14.7"
  instance_class = "db.t3.micro"

  vpc_id     = module.aws_vpc.vpc_id
  subnet_ids = module.aws_vpc.private_subnets
}

## Шаг 4: GCP Infrastructure

In [ ]:
%%writefile modules/gcp/main.tf

# VPC Network
resource "google_compute_network" "vpc" {
  name                    = "multicloud-gcp-vpc"
  auto_create_subnetworks = false
}

resource "google_compute_subnetwork" "private" {
  name          = "private-subnet"
  ip_cidr_range = "10.1.0.0/16"
  region        = var.gcp_region
  network       = google_compute_network.vpc.id
}

# BigQuery Dataset for Analytics
resource "google_bigquery_dataset" "analytics" {
  dataset_id  = "multicloud_analytics"
  location    = var.gcp_region
  description = "Analytics data from AWS and on-premise"

  access {
    role          = "OWNER"
    user_by_email = var.gcp_service_account_email
  }
}

# Cloud Storage for Data Lake
resource "google_storage_bucket" "data_lake" {
  name     = "multicloud-data-lake-${var.gcp_project_id}"
  location = var.gcp_region

  uniform_bucket_level_access = true
  
  lifecycle_rule {
    condition {
      age = 90
    }
    action {
      type          = "SetStorageClass"
      storage_class = "NEARLINE"
    }
  }
}

# Dataflow Job for Processing
resource "google_dataflow_job" "data_processor" {
  name              = "multicloud-data-processor"
  template_gcs_path = "gs://dataflow-templates/latest/Word_Count"
  temp_gcs_location = "gs://${google_storage_bucket.data_lake.name}/temp"
  
  parameters = {
    inputFile  = "gs://${google_storage_bucket.data_lake.name}/input/*"
    output     = "gs://${google_storage_bucket.data_lake.name}/output/"
  }
}

# Cloud Run for ML API
resource "google_cloud_run_service" "ml_api" {
  name     = "ml-prediction-api"
  location = var.gcp_region

  template {
    spec {
      containers {
        image = "gcr.io/${var.gcp_project_id}/ml-api:latest"
        
        env {
          name  = "AWS_DATA_SOURCE"
          value = "http://100.64.1.5:8000/api/data"  # Tailscale IP
        }
      }
    }
  }
}

## Шаг 5: Развертывание инфраструктуры

In [ ]:
# Создание workspaces для разных окружений
!terraform workspace new production
!terraform workspace new staging
!terraform workspace new development

# Переключение на production
!terraform workspace select production

# Инициализация
!terraform init

# План развертывания
!terraform plan -out=multicloud.tfplan

# Применение (раскомментируйте для реального развертывания)
# !terraform apply multicloud.tfplan

## Шаг 6: Настройка Data Pipeline между облаками

In [ ]:
%%writefile data-pipeline/sync-aws-to-gcp.py

import boto3
from google.cloud import storage, bigquery
import pandas as pd

# AWS S3 Client
s3 = boto3.client('s3')

# GCP Clients
gcs_client = storage.Client()
bq_client = bigquery.Client()

def sync_data_aws_to_gcp():
    """
    Синхронизация данных из AWS S3 в GCP BigQuery
    """
    # 1. Скачать данные из S3
    s3_bucket = 'multicloud-data-source'
    s3_key = 'exports/daily_data.csv'
    
    obj = s3.get_object(Bucket=s3_bucket, Key=s3_key)
    df = pd.read_csv(obj['Body'])
    
    # 2. Загрузить в GCS
    gcs_bucket = gcs_client.bucket('multicloud-data-lake')
    blob = gcs_bucket.blob('staging/daily_data.csv')
    blob.upload_from_string(df.to_csv(index=False), content_type='text/csv')
    
    # 3. Загрузить в BigQuery
    table_id = 'multicloud_analytics.daily_metrics'
    
    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.CSV,
        skip_leading_rows=1,
        autodetect=True,
        write_disposition=bigquery.WriteDisposition.WRITE_APPEND
    )
    
    uri = f"gs://multicloud-data-lake/staging/daily_data.csv"
    load_job = bq_client.load_table_from_uri(
        uri, table_id, job_config=job_config
    )
    
    load_job.result()  # Wait for completion
    print(f"✅ Loaded {load_job.output_rows} rows into {table_id}")

if __name__ == "__main__":
    sync_data_aws_to_gcp()

## Шаг 7: Мониторинг Multi-Cloud инфраструктуры

In [ ]:
%%writefile monitoring/prometheus-config.yml

global:
  scrape_interval: 15s
  evaluation_interval: 15s

scrape_configs:
  # AWS Metrics (via CloudWatch Exporter)
  - job_name: 'aws-cloudwatch'
    static_configs:
      - targets: ['100.64.1.5:9106']  # Tailscale IP
    
  # GCP Metrics (via Stackdriver Exporter)
  - job_name: 'gcp-stackdriver'
    static_configs:
      - targets: ['100.64.1.10:9255']  # Tailscale IP
  
  # On-Premise Metrics
  - job_name: 'onprem-node'
    static_configs:
      - targets: ['100.64.1.20:9100']  # Tailscale IP
  
  # Tailscale Network Metrics
  - job_name: 'tailscale'
    static_configs:
      - targets: 
        - '100.64.1.5:9090'   # AWS gateway
        - '100.64.1.10:9090'  # GCP gateway
        - '100.64.1.20:9090'  # On-prem gateway

## Шаг 8: Тестирование связности

In [ ]:
import subprocess
import requests

# Tailscale IPs
TAILSCALE_IPS = {
    'aws': '100.64.1.5',
    'gcp': '100.64.1.10',
    'onprem': '100.64.1.20'
}

def test_connectivity():
    """Проверка связности между облаками через Tailscale"""
    
    print("Testing Multi-Cloud Connectivity...\n")
    
    for name, ip in TAILSCALE_IPS.items():
        # Ping test
        result = subprocess.run(
            ['ping', '-c', '3', ip],
            capture_output=True,
            text=True
        )
        
        if result.returncode == 0:
            print(f"✅ {name.upper()}: Reachable ({ip})")
        else:
            print(f"❌ {name.upper()}: Unreachable ({ip})")
    
    # HTTP connectivity test
    print("\nTesting HTTP Endpoints...\n")
    
    endpoints = {
        'AWS API': f"http://{TAILSCALE_IPS['aws']}:8000/health",
        'GCP ML API': f"http://{TAILSCALE_IPS['gcp']}:8080/health",
        'On-Prem DB': f"http://{TAILSCALE_IPS['onprem']}:5432"
    }
    
    for name, url in endpoints.items():
        try:
            response = requests.get(url, timeout=5)
            print(f"✅ {name}: {response.status_code}")
        except Exception as e:
            print(f"❌ {name}: {str(e)}")

test_connectivity()

## Заключение

### Развернутая Multi-Cloud инфраструктура:

✅ **AWS**: Frontend (S3+CloudFront), API Gateway (ECS), Database (RDS)  
✅ **GCP**: Data Lake (GCS), Analytics (BigQuery), ML API (Cloud Run)  
✅ **On-Premise**: Legacy systems, databases  
✅ **Tailscale Mesh VPN**: Безопасная связь между всеми облаками  
✅ **Unified Monitoring**: Prometheus + Grafana для всех платформ  
✅ **Data Pipeline**: Автоматическая синхронизация данных  

### Преимущества Multi-Cloud подхода:

1. **Избежание vendor lock-in**
2. **Использование лучших сервисов каждого провайдера**
3. **Географическая распределенность**
4. **Disaster recovery и высокая доступность**
5. **Оптимизация затрат**

### Следующие шаги:

1. Настроить автоматическое failover между облаками
2. Добавить Azure в mesh network
3. Реализовать cross-cloud backup strategy
4. Настроить unified logging (ELK Stack)
5. Автоматизировать cost optimization

### Полезные команды:

```bash
# Проверка Tailscale статуса
tailscale status

# Переключение между workspaces
terraform workspace select production

# Просмотр ресурсов в AWS
terraform state list | grep aws

# Просмотр ресурсов в GCP
terraform state list | grep google
```